<a href="https://colab.research.google.com/github/YS-JEOUNG/amazon-review-project/blob/main/code/surprise_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np

!pip install surprise

     |████████████████████████████████| 11.8MB 27.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617611 sha256=80880cba92962077bf94ee48212dea3c6a70c2a07904bfd66150b1b6f2ada8fb
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise import SVD, accuracy
from surprise.model_selection import GridSearchCV, train_test_split

In [ ]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate

# Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/final_data_clean.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ratings,vote,verified,reviewer_id,prod_id,reviewer_name,unix_review_time,prod_name,price,review_text,rating_class,time,clean_text
0,5.0,0,True,A3DKYRF9YGF6A6,6665560953,Amazon Customer,1489363200,You Son of a Bitch! 1987 Embroidered Patch,$6.41,"Five Stars Great quality product, and top notc...",good,2017-03-13,five star great quality product top notch serv...
1,5.0,0,True,A1I2TJ3PQ4WHK1,6665560953,Eric Breneman,1486339200,You Son of a Bitch! 1987 Embroidered Patch,$6.41,don't push too many pencils details on this pa...,good,2017-02-06,not push many pencil detail patch good def fav...
2,5.0,0,True,A1ZDJ2BYZXZHLQ,6665560953,Kirkja,1482451200,You Son of a Bitch! 1987 Embroidered Patch,$6.41,Great deal if you buy in bulk! Bought these as...,good,2016-12-23,great deal buy bulk buy gift friends really im...
3,5.0,0,True,A1RW3PF3UTARAV,6665560953,Brian K.,1481673600,You Son of a Bitch! 1987 Embroidered Patch,$6.41,It's made extremely well and looks GREAT! I've...,good,2016-12-14,make extrehemely well look great see similar p...
4,5.0,9,True,APVFJYDRTYNK5,886293162X,Mrs. Sally L. C. Marsh,1290816000,"Moleskine Volant Address Book, Large, Magenta,...",NaN,"Nice and perfect for an address book Thin, you...",good,2010-11-27,nice perfect address book thin could slide dra...


In [ ]:
# 원하는 품목을 입력하면 데이터 프레임에서 품목만 찾아주는 함수
def word_select(word):
  words = [word.lower(), word.capitalize(), word.lower()+'s', word.capitalize()+'s']
  review_df_x = df[df['prod_name'].str.contains('|'.join(words))]

  return review_df_x

In [ ]:
df[df['prod_name'].isnull()]

,ratings,vote,verified,reviewer_id,prod_id,reviewer_name,unix_review_time,prod_name,price,review_text,rating_class,time,clean_text
2182926,5.0,0,True,A3I9D3CMM362H0,B00DV92LRM,Catherine Kraft,1487203200,NaN,NaN,Love this die! Love this die. Makes these flow...,good,2017-02-16,love die love die make flower much easier make...
2376582,5.0,0,True,A323CN1N6FCZKL,B00N2TWI5G,K Cox,1456272000,NaN,$28.32,Five Stars Love these templates! Best custome...,good,2016-02-24,five star love tehemplates best customer servi...
2376583,5.0,0,False,A2WD7YVEJWHYY2,B00N2TWI5G,Nora A.,1415664000,NaN,$28.32,Five Stars Excellent product. Cordial owners. ...,good,2014-11-11,five star excellent product cordial owners wel...


In [ ]:
df = df.drop([2182926, 2376582, 2376583])

In [ ]:
df[df['prod_name'].isnull()]

,ratings,vote,verified,reviewer_id,prod_id,reviewer_name,unix_review_time,prod_name,price,review_text,rating_class,time,clean_text


In [ ]:
df = df[['reviewer_id', 'prod_id', 'ratings']]

In [ ]:
df.head()

,reviewer_id,prod_id,ratings
0,A3DKYRF9YGF6A6,6665560953,5.0
1,A1I2TJ3PQ4WHK1,6665560953,5.0
2,A1ZDJ2BYZXZHLQ,6665560953,5.0
3,A1RW3PF3UTARAV,6665560953,5.0
4,APVFJYDRTYNK5,886293162X,5.0


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewer_id', 'prod_id', 'ratings']], reader)

# SVD

In [ ]:
trainset, testset = train_test_split(data, test_size=0.1)

# SVD 알고리즘 피팅
algo_svd = SVD(n_epochs=50, lr_all=0.01, reg_all=0.2)
algo_svd.fit(trainset)

In [ ]:
predictions = algo_svd.test(testset)
accuracy.rmse(predictions)
# RMSE: 1.1256

RMSE: 1.1256


1.1255913888983322

In [ ]:
# 가상의 id
uid = 'A18C6CLQ8LQKEF'
iid = 'B000WLEL84'
algo_svd.predict(uid, iid)

Prediction(uid='A18C6CLQ8LQKEF', iid='B000WLEL84', r_ui=None, est=4.315948915294567, details={'was_impossible': False})

In [ ]:
# UID를 인풋으로 받고 상품을 추천하는 방법은?
# 실험대상: 'A1RW3PF3UTARAV' (실제 reviewer_id 중 아무거나 뽑음)
product_id = df['prod_id'].unique() # np.ndarray 오브젝트, len=6665
# 사용자 리뷰를 빼도 product_id 개수는 동일 (생각해보면 당연한 소리... 그래도 안 겹치게 빼 줍시다)
product_id = df[df['reviewer_id'] != 'A1RW3PF3UTARAV']['prod_id'].unique()

In [ ]:
uid = 'A1RW3PF3UTARAV'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svd.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B00D6D5ADA
B00NC56S06
B00OPKOXOU
B010O7O6BW
B00008XPCF
B000KNP7X2
B000OVK1P4
B00153AMEA
B001JPF3ZM
B001JPMV9S
B001M0E5YO
B001NVD5O8
B001U36A82
B001VB4MME
B002R8G4YO
B002RBB0RW
B003AVEROQ
B003CU3VEM
B003I2HUOQ
B0044SB47C


In [ ]:
product_id = df['prod_id'].unique() 
product_id = df[df['reviewer_id'] != 'A3DKYRF9YGF6A6']['prod_id'].unique()

In [ ]:
uid = 'A3DKYRF9YGF6A6'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svd.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B003SBLIQS
B0017J68WM
B001M0E5YO
B001U36A82
B001VB4MME
B002R8G4YO
B0032643WA
B003I2HUOQ
B0046VFGYO
B004CYFJ5G
B004LE5I4O
B004LE2KKY
B004UWWL0G
B00557D8WU
B005HZ2574
B006WRZEFG
B008VQ25MQ
B0090VGMGQ
B00936MGRW
B009R7JG2A


In [ ]:
product_id = df['prod_id'].unique()
product_id = df[df['reviewer_id'] != 'A1I2TJ3PQ4WHK1']['prod_id'].unique()

In [ ]:
uid = 'A1I2TJ3PQ4WHK1'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svd.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B004P3W2SG
B00VGU9F7C
B00YLNLCDG
B000BRMZAK
B000I6F9AW
B000U0DYVM
B001D6KOHO
B001EU0RQ2
B001VB4MME
B002NFK72G
B002R8G4YO
B003ZSDDYE
B0045D1GGU
B004AGOXEO
B004CYFJ5G
B004LE5I4O
B004LE2KKY
B004OLN1VG
B004UWWL0G
B004WBY2TI


#  SVDpp

In [ ]:
# SVD 활용하는 그리드서치CV만들기
# 그리드서치에서 쓸 파라미터 dict 정의
param_grid = {
    "n_epochs": [5, 10, 20, 50],
    "lr_all": [0.001, 0.002, 0.005, 0.01],
    "reg_all": [0.2, 0.4, 0.6]
}
# GridSearchCV
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
# gs 피팅
gs.fit(data)
print(gs.best_score["rmse"]), print(gs.best_params["rmse"])
print(gs.best_score["mae"]), print(gs.best_params["mae"])
# rmse: 1.08, n_epochs 50, lr_all 0.01, reg_all 0.2
# mae: 0.77, n_epochs 50, lr_all 0.01, reg_all 0.2

In [ ]:
trainset, testset = train_test_split(data, test_size=0.1)

# SVD 알고리즘 피팅
algo_svdpp = SVDpp(n_epochs=50, lr_all=0.01, reg_all=0.2)
algo_svdpp.fit(trainset)

In [ ]:
predictions = algo_svdpp.test(testset)
accuracy.rmse(predictions)
# RMSE: 1.1286

RMSE: 1.1286


1.1285676260114093

In [ ]:
# 가상의 id
uid = 'A18C6CLQ8LQKEF'
iid = 'B000WLEL84'
algo_svdpp.predict(uid, iid)

Prediction(uid='A18C6CLQ8LQKEF', iid='B000WLEL84', r_ui=None, est=4.315250013463959, details={'was_impossible': False})

In [ ]:
# UID를 인풋으로 받고 상품을 추천하는 방법은?
# 실험대상: 'A1RW3PF3UTARAV' (실제 reviewer_id 중 아무거나 뽑음)
product_id = df['prod_id'].unique() # np.ndarray 오브젝트, len=6665
# 사용자 리뷰를 빼도 product_id 개수는 동일 (생각해보면 당연한 소리... 그래도 안 겹치게 빼 줍시다)
product_id = df[df['reviewer_id'] != 'A1RW3PF3UTARAV']['prod_id'].unique()

In [ ]:
uid = 'A1RW3PF3UTARAV'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svdpp.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B00BLO19NQ
B00VGU9F7C
B01DIBSCRA
B01G2IIYCY
B0007QO7BK
B000FPYPUQ
B000I6F9AW
B000KNP7X2
B000W88U9S
B000YQH486
B000YZ7EK0
B00114OQPY
B0015AOL2C
B0017J68WM
B0017RGKAO
B0018HU73E
B001CE8J9M
B001D6KOHO
B001EU0RQ2
B001GSHX14


In [ ]:
product_id = df['prod_id'].unique() 
product_id = df[df['reviewer_id'] != 'A3DKYRF9YGF6A6']['prod_id'].unique()

In [ ]:
uid = 'A3DKYRF9YGF6A6'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svdpp.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B004P3W2SG
B00MQ95ELS
B00P0WU3J6
B00VGU9F7C
B01G2IIYCY
B000I6F9AW
B000KNP7X2
B000SN170G
B000U4CZJK
B000W88U9S
B000YZ7EK0
B00114OOZQ
B00114OQPY
B0017J68WM
B0018HU73E
B001D6KOHO
B001EU0RQ2
B001GSHX14
B001UHNI3I
B001WOKEAE


In [ ]:
product_id = df['prod_id'].unique()
product_id = df[df['reviewer_id'] != 'A1I2TJ3PQ4WHK1']['prod_id'].unique()

In [ ]:
uid = 'A1I2TJ3PQ4WHK1'
pred_list = []
top_items = 20
for iid in product_id:
    pred = algo_svdpp.predict(uid, iid)
    pred_list.append((pred[1], pred[3]))
def takeSecond(elem):
    return elem[1]
pred_list.sort(reverse=True, key=takeSecond)
reclist = pred_list[:top_items]
for i in range(top_items):
    print(reclist[i][0])

B000YQCWU6
B000YZ7JAA
B002FCBS20
B005UXIMPC
B0069COU4A
B009444TNM
B00BOG0JK0
B00J41VKM8
B00K13898G
B00MQ95ELS
B00P0WU3J6
B00VGU9F7C
B010O7O6BW
B01AVNW1KW
B01G2IIYCY
B000I6F9AW
B000KNP7X2
B000W88U9S
B00114OOZQ
B0015AOL2C
